# Redes Neurais Artificiais 2025.1

- **Disciplina**: Redes Neurais Artificiais 2025.1
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
- **Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta parte do Projeto Prático diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você deseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

**Importação de bibliotecas**

In [1]:
import prettytable
import numpy as np
import pandas as pd
import json
import random
import joblib
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

**Importação do dataset**

In [2]:
file_path = "covtype.csv"
pd.set_option('display.max_columns', None)

df = pd.read_csv(file_path) #Dataset com atributos categóricos
categoric_cols = [col for col in df.columns if 'Soil_Type' in col or 'Wilderness_Area' in col]
df = df.drop(columns=categoric_cols)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,5
1,2590,56,2,212,-6,390,220,235,151,6225,5
2,2804,139,9,268,65,3180,234,238,135,6121,2
3,2785,155,18,242,118,3090,238,238,122,6211,2
4,2595,45,2,153,-1,391,220,234,150,6172,5


**Importação das melhores arquiteturas**

In [3]:
with open('resultado_melhores_arquiteturas.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

pd.DataFrame(data)

,camadas,solver,activation,iterations,media_acuracia,desvio_acuracia,media_f1,desvio_f1
0,"[48, 60]",sgd,relu,150,0.807700,0.002632,0.804039,0.002963
1,"[36, 28]",adam,logistic,150,0.798396,0.002396,0.795103,0.002576
2,"[32, 32]",sgd,relu,150,0.781922,0.003023,0.777013,0.003404
3,"[17, 8]",adam,logistic,100,0.741158,0.003078,0.732777,0.003360
4,"[12, 13]",adam,relu,100,0.741100,0.003800,0.734000,0.004000
5,"[10, 15]",adam,relu,100,0.739800,0.003800,0.732400,0.004500


In [4]:
table = prettytable.PrettyTable(['Neurônios Ocultos', 'Função de Ativação', 'Utilização de Atributos Categóricos', 'Desempenho Médio', 'Número de Repetições'])
arquiteturas = []

for arquitetura in data:
    table.add_row([tuple(arquitetura['camadas']), 
                  arquitetura['activation'],
                   "Não",
                  f"{arquitetura['media_f1']:.4f} +- {arquitetura['desvio_f1']:.4f}", 
                  100])
    arquiteturas.append(
        [tuple(arquitetura['camadas']),
         arquitetura['solver'],
         arquitetura['activation'],
         arquitetura['iterations']
        ])

print(table)

+-------------------+--------------------+-------------------------------------+------------------+----------------------+
| Neurônios Ocultos | Função de Ativação | Utilização de Atributos Categóricos | Desempenho Médio | Número de Repetições |
+-------------------+--------------------+-------------------------------------+------------------+----------------------+
|      (48, 60)     |        relu        |                 Não                 | 0.8040 +- 0.0030 |         100          |
|      (36, 28)     |      logistic      |                 Não                 | 0.7951 +- 0.0026 |         100          |
|      (32, 32)     |        relu        |                 Não                 | 0.7770 +- 0.0034 |         100          |
|      (17, 8)      |      logistic      |                 Não                 | 0.7328 +- 0.0034 |         100          |
|      (12, 13)     |        relu        |                 Não                 | 0.7340 +- 0.0040 |         100          |
|      (10, 15) 

In [5]:
arquiteturas

[[(48, 60), 'sgd', 'relu', 150],
 [(36, 28), 'adam', 'logistic', 150],
 [(32, 32), 'sgd', 'relu', 150],
 [(17, 8), 'adam', 'logistic', 100],
 [(12, 13), 'adam', 'relu', 100],
 [(10, 15), 'adam', 'relu', 100]]

In [6]:
y = df['Cover_Type']
X = df.drop(columns=['Cover_Type'])

**Definindo parâmetros e Hiperparâmetros a serem otimizados**

In [7]:
parametros = {
    'solver' :  ['adam', 'sgd'], 
    'batch_size' : [100, 200],
    'learning_rate_init' : [0.0001, 0.001, 0.01],
    'n_iter_no_change' : [10, 20], #paciência
    'max_iter' : [100, 150] #número de épocas
}

## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

In [8]:
def busca_grade(mlp, parameters, scorers, X, y):
    busca = GridSearchCV(mlp, parameters, scoring=scorers, n_jobs=-1,  verbose=2, refit='f1')
    busca.fit(X,y)
    return busca.cv_results_, busca.best_params_, busca.best_score_

Tal qual a parte anterior deste projeto, as execuções foram feitas em dispositivos distintos, cada modelo foi obtido conforme a seguinte célula:

In [9]:
model = arquiteturas[5]
print(model)
mlp = MLPClassifier(hidden_layer_sizes = model[0], solver=model[1], activation=model[2], max_iter=model[3])

[(10, 15), 'adam', 'relu', 100]


In [10]:
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'f1': make_scorer(f1_score, average='weighted')  
}

As execuções são excutadas da seguinte maneira:

In [28]:
# resultado, melhor_parametros, melhor_f1 = busca_grade(mlp, parametros,scorers, X, y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


KeyboardInterrupt: 

Os resultados são armazenados da seguinte maneira:

In [ ]:
# resultado = pd.DataFrame(resultado).to_dict()

In [ ]:
# melhor_parametros['f1_score'] = melhor_f1

In [ ]:
# melhor_parametros

In [ ]:
# with open("/kaggle/working/resultados_arquitetura3.json", "w", encoding="utf-8") as f:
    # json.dump(resultado, f, ensure_ascii=False, indent=4)

In [ ]:
# with open("/kaggle/working/melhor_arquitetura3.json", "w", encoding="utf-8") as f:
    # json.dump(melhor_parametros, f, ensure_ascii=False, indent=4)

## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto

**Recuperando melhores modelos das buscas em grade**

Obs: Por limitações de processamento e tempo, não foi possível obter o resultado da busca em grade da arquitetura [(48, 60), 'sgd', 'relu', 150]]

In [43]:
melhores_modelos = []
for i in range(6):
    if i==0:
        continue
        
    with open(f"buscas_em_grade/melhor_arquitetura{i}.json", 'r', encoding='utf-8') as f:
        data = json.load(f)
        data['hidden_layer_sizes'] = arquiteturas[i][0]
        melhores_modelos.append(data)

melhores_modelos = pd.DataFrame(melhores_modelos).sort_values(by='f1_score', ascending=False, ignore_index=True)
melhores_modelos

,batch_size,learning_rate_init,max_iter,n_iter_no_change,solver,f1_score,hidden_layer_sizes
0,200,0.0100,150,20,adam,0.651257,"(10, 15)"
1,200,0.0100,150,20,adam,0.632167,"(32, 32)"
2,200,0.0001,150,10,adam,0.621328,"(12, 13)"
3,200,0.0001,100,10,adam,0.598039,"(17, 8)"
4,200,0.0001,100,20,adam,0.579949,"(36, 28)"


In [50]:
parametros_finais = melhores_modelos.loc[0]
parametros_finais = parametros_finais.drop('f1_score')
batch = int(parametros_finais['batch_size'])
lri = float(parametros_finais['learning_rate_init'])
epochs = int(parametros_finais['max_iter'])
solver = parametros_finais['solver']
layers = tuple(parametros_finais['hidden_layer_sizes'])
patience = parametros_finais['n_iter_no_change']
melhor_modelo = MLPClassifier(hidden_layer_sizes=layers, batch_size=batch, solver=solver, max_iter=epochs, learning_rate_init=lri, n_iter_no_change=patience, verbose=True)
melhor_modelo

MLPClassifier(batch_size=200, hidden_layer_sizes=(10, 15),
              learning_rate_init=0.01, max_iter=150,
              n_iter_no_change=np.int64(20), verbose=True)

## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html  
2. Treine o modelo com todos os dados  
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas

**Treinando do melhor modelo com todos os dados**

In [45]:
df_completo = pd.read_csv(file_path)

In [46]:
X_completo = df_completo.drop('Cover_Type', axis=1)
y_completo = df_completo['Cover_Type']

In [47]:
# Verificar tipos e ausência de nulos
assert not X_completo.isnull().any().any(), "X tem valores nulos!"
assert not y_completo.isnull().any(), "y tem valores nulos!"
assert all([dtype == 'int64' for dtype in X_completo.dtypes]), "X tem colunas que não são int64!"

In [48]:
# Escalar X
scaler = StandardScaler()
X_std = scaler.fit_transform(X_completo)

In [51]:
melhor_modelo.fit(X_std, y_completo)

Iteration 1, loss = 0.61679529
Iteration 2, loss = 0.56945147
Iteration 3, loss = 0.55851083
Iteration 4, loss = 0.55282780
Iteration 5, loss = 0.54926068
Iteration 6, loss = 0.54652560
Iteration 7, loss = 0.54402596
Iteration 8, loss = 0.54189775
Iteration 9, loss = 0.54133630
Iteration 10, loss = 0.54022873
Iteration 11, loss = 0.53976433
Iteration 12, loss = 0.53818309
Iteration 13, loss = 0.53723846
Iteration 14, loss = 0.53651020
Iteration 15, loss = 0.53577049
Iteration 16, loss = 0.53486676
Iteration 17, loss = 0.53403546
Iteration 18, loss = 0.53421135
Iteration 19, loss = 0.53397822
Iteration 20, loss = 0.53363577
Iteration 21, loss = 0.53398530
Iteration 22, loss = 0.53313337
Iteration 23, loss = 0.53257373
Iteration 24, loss = 0.53267491
Iteration 25, loss = 0.53206887
Iteration 26, loss = 0.53226120
Iteration 27, loss = 0.53224057
Iteration 28, loss = 0.53247240
Iteration 29, loss = 0.53204256
Iteration 30, loss = 0.53145066
Iteration 31, loss = 0.53043242
Iteration 32, los

MLPClassifier(batch_size=200, hidden_layer_sizes=(10, 15),
              learning_rate_init=0.01, max_iter=150,
              n_iter_no_change=np.int64(20), verbose=True)

**Armazenamento do modelo**

In [55]:
joblib.dump(melhor_modelo, 'mlp_model.joblib')

['mlp_model.joblib']

**Recuperando e executando previsão com o modelo**

In [60]:
def recuperar_modelo():
    return joblib.load('mlp_model.joblib')

In [69]:
linha_teste = df_completo.iloc[[500]] 
teste = scaler.transform(linha_teste.drop('Cover_Type', axis=1))
teste

array([[-1.22280094, -1.25683363, -0.01384886, -1.12646036, -0.79627259,
        -0.44774413, -0.19223289, -0.62314308, -0.09218313, -0.45106019,
         1.10807983, -0.23285905, -0.87936402, -0.26067295, -0.07241628,
        -0.11454904, -0.09149053, -0.14764925, -0.05249979, -0.10698593,
        -0.0134444 , -0.017555  , -0.04447524, -0.24394681, -0.14773442,
        -0.23321617, -0.17586644, -0.03212513, -0.00227232, -0.07014787,
        -0.07697154, -0.05726389, -0.08348001, -0.12725587, -0.03800519,
        -0.24685984, -0.33221921, -0.19497304, -0.02857418, -0.06690264,
        -0.04327416, -0.04038377,  2.01033582, -0.2340314 , -0.21497961,
        -0.31523803, -0.2902841 , -0.05273005, -0.05714275, -0.01431283,
        -0.02265307, -0.16595612, -0.15601398, -0.12365355]])

In [72]:
linha_teste['Cover_Type']

500    2
Name: Cover_Type, dtype: int64

In [70]:
rna = recuperar_modelo()

In [71]:
rna.predict(teste)

array([2])